In [79]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Ejercicio 1

In [80]:
#path = r'C:\Users\ezequ\OneDrive\Documentos\Facultad\Aprendizaje Automatico\material-main\tp\01_aprendizaje_supervisado\datos\data.csv'
#df = pd.read_csv(path)
df = pd.read_csv('data.csv')

In [81]:
df['target'].value_counts()

target
0    343
1    157
Name: count, dtype: int64

La relacion entre clases es 157/343 = 0.46 siendo la mayoritaria la clase representada por el 0 por lo que estamos ante un dataset desbalanceado

A priori hacer un split 400 100 para desarrollo validacion suena razonable , estamos dejando 20% de nuestro dataset para validar.

Capaz es un poco mucho ese porcentaje pero creo que eso aseguraria que los resultados obtenidos tendran un poco mas de robustez.

Dentro de desarrollo capaz podemos hacer algo como k folds con k = 4 para sacarle un poco mas el jugo a los datos.

train = 80%
validacion = 20%

In [82]:
def train_test_split(df,train_size=0.8):
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    n = df.shape[0]
    cantNeg, cantPos = df['target'].value_counts()
    proporcion = cantPos / n

    cant_en_train = int(n*train_size)
    cant_en_validacion = n - cant_en_train

    res = []
    cant_positivas = int(proporcion * cant_en_validacion)
    cant_negativas = cant_en_validacion - cant_positivas

    contador_positivos, contador_negativos = 0, 0
    for index, value in enumerate(df['target']):
        if value == 0:
            contador_negativos += 1
            res.append(index)

        if contador_negativos == cant_negativas:
            break
    for index, value in enumerate(df['target']):

        if value == 1:
            contador_positivos += 1
            res.append(index)

        if contador_positivos == cant_positivas:
            break

    df_dev = df.loc[~df.index.isin(res)].reset_index(drop=True)
    df_val = df.loc[res].reset_index(drop=True)
    return df_dev,df_val

In [83]:
df_dev , df_val =  train_test_split(df)


# Ejercicio 2

In [84]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, roc_auc_score , average_precision_score
from sklearn.model_selection import cross_val_predict


In [85]:
X_train = df_dev.drop(columns=['target']).values

y_train = df_dev['target'].values

X_val   = df_val.drop(columns=['target']).values

y_val   =  df_val['target'].values

In [86]:
tree_clf = DecisionTreeClassifier(max_depth=3, random_state=51)
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=51)

In [87]:

scoring = ['accuracy', 'roc_auc', 'average_precision']

skf = StratifiedKFold(n_splits=5)

cv_results = cross_validate(tree_clf, X_train, y_train, cv = skf, scoring = scoring,return_train_score = True)

predicted_labels = cross_val_predict(tree_clf, X_train, y_train, cv = skf)

metricas_globales = {'Accuracy Global' : accuracy_score(y_train, predicted_labels),
                     'Auc Global' : roc_auc_score(y_train, predicted_labels),
                      'Average Precision Global' : average_precision_score(y_train, predicted_labels)
}

cv_results.update(metricas_globales)

df_resultados_ej_2_2 = pd.DataFrame({key: cv_results[key] for key in list(cv_results.keys())[3:]})

df_resultados_ej_2_2.index = df_resultados_ej_2_2.index + 1 # asi el indice del df se correponde con el k


In [88]:
df_resultados_ej_2_2

,train_accuracy,test_roc_auc,train_roc_auc,test_average_precision,train_average_precision,Accuracy Global,Auc Global,Average Precision Global
1,0.834375,0.585470,0.770614,0.432637,0.656625,0.695,0.599467,0.386459
2,0.818750,0.732000,0.852706,0.484188,0.713912,0.695,0.599467,0.386459
3,0.825000,0.613455,0.796826,0.404476,0.657074,0.695,0.599467,0.386459
4,0.809375,0.596000,0.854492,0.394547,0.713593,0.695,0.599467,0.386459
5,0.837500,0.705455,0.836928,0.516564,0.733381,0.695,0.599467,0.386459


In [89]:
cv_results.keys()

dict_keys(['fit_time', 'score_time', 'test_accuracy', 'train_accuracy', 'test_roc_auc', 'train_roc_auc', 'test_average_precision', 'train_average_precision', 'Accuracy Global', 'Auc Global', 'Average Precision Global'])

Basado en pagina 288 de Introduction to Machine Learning with Python

In [90]:
alturas = [3, 5, None]
criterios = ["gini", "entropy"]

resultados = {}

for altura in alturas:
    for criterio in criterios:

        tree = DecisionTreeClassifier(max_depth = altura, criterion = criterio, random_state = 51)

        cv_results_grid = cross_validate(tree, X_train, y_train, cv = skf, scoring = 'accuracy', return_train_score = True)

        accuracy_train = cv_results_grid['train_score'].mean()

        accuracy_test =  cv_results_grid['test_score'].mean()



        resultados[(altura, criterio)] = (accuracy_train,accuracy_test)

results_df = pd.DataFrame(resultados.values(), index=resultados.keys(), columns=['Train Score', 'Test Score'])



In [91]:
results_df

Train Score  Test Score
3   gini        0.825000       0.695
    entropy     0.786250       0.670
5   gini        0.930000       0.690
    entropy     0.920625       0.655
NaN gini        1.000000       0.680
    entropy     1.000000       0.645

# Ejercicio 3

In [92]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
import scipy

# Hay que revisar en la documentacion del modelo cada parametro y usar una distribucion con los valores mas probables
# Las listas son interpretadas como distribuciones uniformes discretas

# Comento los parámetros que generan errores
dist_params = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_depth': scipy.stats.randint(1,40), #puede ser float
    'min_samples_split': scipy.stats.randint(2,20), #puede ser float
    'min_samples_leaf': scipy.stats.randint(1,20),
    #'min_weight_fraction_leaf':scipy.stats.uniform(0,1) ,
    'max_features': scipy.stats.randint(1,20),
    'random_state': [42],
    'max_leaf_nodes': scipy.stats.randint(2,len(X_train)),
    #'min_impurity_decrease': scipy.stats.uniform(0,0.1),
    #'class_weight': {i: scipy.stats.uniform(0,1) for i in range(len(X_train[0]))},
    #'ccp_alpha': scipy.stats.uniform(0,0.1),
}

tree = DecisionTreeClassifier()
n_iter_search = 50    # Esto es la cantidad de configuraciones que pruebo
random_search = RandomizedSearchCV(estimator=tree, param_distributions=dist_params, scoring='roc_auc', n_iter=n_iter_search)
random_search.fit(X_train, y_train)

mejor_modelo = random_search.best_estimator_
mejor_score = random_search.best_score_

print("Mejor modelo:", mejor_modelo)
print("Mejor score AUCROC:", mejor_score)

Mejor modelo: DecisionTreeClassifier(criterion='entropy', max_depth=28, max_features=15,
                       max_leaf_nodes=90, min_samples_leaf=17,
                       min_samples_split=13, random_state=42)
Mejor score AUCROC: 0.6512773374773374


In [75]:
from sklearn.neighbors import KNeighborsClassifier

dist_params = {
    'n_neighbors': scipy.stats.randint(1, 30),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': scipy.stats.randint(1, 50),
    'p': scipy.stats.randint(1, 5),
    'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski'],
    #'metric_params':,
    #'n_jobs':,
}

knn = KNeighborsClassifier()
n_iter_search = 50
random_search = RandomizedSearchCV(estimator=knn, param_distributions=dist_params, scoring='roc_auc', n_iter=n_iter_search)

# Normalizo los datos para knn
from sklearn import preprocessing
X_train_normalized = preprocessing.normalize(X_train.T).T
random_search.fit(X_train_normalized, y_train)

mejor_modelo = random_search.best_estimator_
mejor_score = random_search.best_score_

print("Mejor modelo:", mejor_modelo)
print("Mejor score AUCROC:", mejor_score)


Mejor modelo: KNeighborsClassifier(algorithm='brute', leaf_size=44, n_neighbors=29,
                     weights='distance')
Mejor score AUCROC: 0.7948595700595702


In [76]:
from sklearn.svm import SVC

dist_params = {
    'C': scipy.stats.uniform(0, 10),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': scipy.stats.randint(1, 6),
    'gamma': ['scale', 'auto' ],
    'coef0': scipy.stats.uniform(0, 1),
    'shrinking': [True, False],
    'probability': [True, False],
    'tol': scipy.stats.uniform(0, 0.1),
    'cache_size': scipy.stats.randint(1, 1000),
    #'class_weight': [{0: x, 1: 1 - x} for x in np.linspace(0, 1, 10)],
}

svm = SVC()
n_iter_search = 50
random_search = RandomizedSearchCV(estimator=svm, param_distributions=dist_params, scoring='roc_auc', n_iter=n_iter_search)
random_search.fit(X_train, y_train)

mejor_modelo = random_search.best_estimator_
mejor_score = random_search.best_score_

print("Mejor modelo:", mejor_modelo)
print("Mejor score AUCROC::", mejor_score)

Mejor modelo: SVC(C=3.383345113104297, cache_size=175, coef0=0.040189738419383114, degree=4,
    probability=True, tol=0.047621618380040476)
Mejor score AUCROC:: 0.8815499611499611


In [77]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Hubo bastantes errores con los parámetros
dist_params = {
    #'solver': ['svd','lsqr', 'eigen'],
    #'shrinkage': ['auto',None], #scipy.stats.uniform(0, 1),
    #'priors': [],
    #'n_components': scipy.stats.randint(1, min(len(X_train[0,:]), len(X_train[0]-1))),
    'tol': scipy.stats.uniform(0, 0.1)
    #'covariance_estimator':
}

lda = LinearDiscriminantAnalysis()
n_iter_search = 50
random_search = RandomizedSearchCV(estimator=lda, param_distributions=dist_params, scoring='roc_auc', n_iter=n_iter_search)
random_search.fit(X_train, y_train)

mejor_modelo = random_search.best_estimator_
mejor_score = random_search.best_score_

print("Mejor modelo:", mejor_modelo)
print("Mejor score AUCROC::", mejor_score)

Mejor modelo: LinearDiscriminantAnalysis(tol=0.041285137484439564)
Mejor score AUCROC:: 0.7554754726754727


In [78]:
from sklearn.naive_bayes import GaussianNB

dist_params = {
    'var_smoothing': scipy.stats.uniform(0, 1),
    'priors': [None, [0.5, 0.5]],
}

nb = GaussianNB()
n_iter_search = 50
random_search = RandomizedSearchCV(estimator=nb, param_distributions=dist_params, scoring='roc_auc', n_iter=n_iter_search)
random_search.fit(X_train, y_train)

mejor_modelo = random_search.best_estimator_
mejor_score = random_search.best_score_

print("Mejor modelo:", mejor_modelo)
print("Mejor score AUCROC:", mejor_score)

Mejor modelo: GaussianNB(priors=[0.5, 0.5], var_smoothing=0.005040613463469068)
Mejor score AUCROC: 0.8578516446516445
